![sandcastle banner image](./sandcastle_intro_banner.png)

In [1]:
# this is the bash command to remove your virtual environment

# sudo /anaconda/bin/conda remove -n py38_selenium --all

In [2]:
# these are the commands to create the coding environment for this notebook

# sudo /anaconda/bin/conda create -y --name py38_selenium python=3.8
# conda activate /anaconda/envs/py38_selenium
# pip install ipykernel selenium webdriver-manager beautifulsoup4 lxml html5lib xlrd wget pandas holidays

In [3]:
# these commands are for installing google-chrome on ubuntu if needed

# wget -nc https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
# sudo apt update
# sudo apt install -f ./google-chrome-stable_current_amd64.deb

In [4]:
# check installed version
!chromium-browser --version

Chromium 118.0.5993.88 snap


check the version based on Release
https://chromedriver.chromium.org/downloads

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime
from holidays import Belgium
from time import sleep
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import math
import requests

### selenium v4

Since version 4 you need to use Chrome Driver Manager and it will automatically  
install the correct Chrome Driver for your environment.

In [6]:
! pip show selenium

Name: selenium
Version: 4.8.3
Summary: 
Home-page: https://www.selenium.dev
Author: 
Author-email: 
License: Apache 2.0
Location: /anaconda/envs/py38_selenium/lib/python3.8/site-packages
Requires: certifi, trio, trio-websocket, urllib3
Required-by: 


In [7]:
print ('Last testrun on: ' + datetime.now().strftime("%d %b %Y"))

Last testrun on: 21 Oct 2023


In [8]:
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--window-size=1920,1200")  # adviced to increase resolution

# here is where the magic happens
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [9]:
# open the webpage where we can find all tides
driver.get("https://odnature.naturalsciences.be/marine-forecasting-centre/nl/harmonic-tides")

In [10]:
# maximize the window
driver.maximize_window()

In [11]:
# check the image (this is used for debugging code)
driver.save_screenshot('screen.png')

True

## change the start date

In [12]:
# Change the start date formaat JJJJ-MM-DD
begindatum = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[id='harmonic-start-date']")))

begindatum.clear() # Clear any existing value
begindatum.send_keys("2024-01-01") # CHANGE THE YEAR

## change the end date

In [13]:
# Locate the end date input field
einddatum = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "input[id='harmonic-end-date']")))

# Click the input field to make it editable
einddatum.click()

einddatum.clear() # Clear any existing value
einddatum.send_keys("2024-12-31") # CHANGE THE YEAR

### downloading website table

In [14]:
button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "update-table")))
button.click()

In [15]:
# wait 5 seconds, for the webpage to load
sleep(5)

In [16]:
# Get the page source after all interactions
page_source = driver.page_source

In [17]:
# Parse the page source with BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

In [18]:
# Extract the table with the specific attribute name="tableResults"
table = soup.find('table', {'name': 'tableResults'})

In [19]:
# Convert the table to a DataFrame
df = pd.read_html(str(table))[0]

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 733 entries, 0 to 732
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Datum               732 non-null    object
 1   Hoog water [m TAW]  732 non-null    object
 2   Tijd [UTC]          732 non-null    object
 3   Laag water [m TAW]  732 non-null    object
 4   Tijd [UTC].1        732 non-null    object
dtypes: object(5)
memory usage: 28.8+ KB


In [21]:
df

,Datum,Hoog water [m TAW],Tijd [UTC],Laag water [m TAW],Tijd [UTC].1
0,NaN,NaN,NaN,NaN,NaN
1,2024-01-01,4.17,03:11,0.53,09:49
2,2024-01-01,4.17,15:35,0.99,21:49
3,2024-01-02,4.01,03:46,0.68,10:30
4,2024-01-02,4.01,16:14,1.11,22:31
...,...,...,...,...,...
728,2024-12-29,4.15,23:34,0.73,17:24
729,2024-12-30,4.27,11:42,0.63,05:55
730,2024-12-30,--.--,--.--,0.57,18:04
731,2024-12-31,4.33,00:11,0.42,06:34


### clean and prepare the data

In [22]:
# change the column names
df.columns = ['datum', 'hoog_water', 'hoog_tijd', 'laag_water', 'laag_tijd']

In [23]:
# remove record 0 because it is all NaN
df = df.drop(0, axis=0)

In [24]:
# first replace the values "--.--"" into NaN
x = {"--.--":np.nan}
df = df.replace(x)

# change hoog en laag water from string into float numbers
df = df.astype({'hoog_water':'float', 'laag_water':'float'})

In [25]:
# Convert 'datum' to datetime format
df['datum'] = pd.to_datetime(df['datum'])

In [26]:
# Combine the date from 'datum' with the time values from 'hoog_tijd' and 'laag_tijd'
df['hoog_tijd'] = pd.to_datetime(df['datum'].dt.strftime('%Y-%m-%d') + ' ' + df['hoog_tijd'])
df['laag_tijd'] = pd.to_datetime(df['datum'].dt.strftime('%Y-%m-%d') + ' ' + df['laag_tijd'])

In [27]:
# Localize to UTC
df['hoog_tijd'] = df['hoog_tijd'].dt.tz_localize('UTC')
df['laag_tijd'] = df['laag_tijd'].dt.tz_localize('UTC')

In [28]:
# Converteer de tijden naar de lokale tijdzone van Oostende, België ("Europe/Brussels")
df['laag_tijd'] = df['laag_tijd'].dt.tz_convert('Europe/Brussels')
df['hoog_tijd'] = df['hoog_tijd'].dt.tz_convert('Europe/Brussels')

In [29]:
# remove the column "datum"
df.drop(["datum"], axis=1, inplace=True)

In [30]:
# Get basic statistics for each column
print(df.describe())

       hoog_water  laag_water
count  707.000000  707.000000
mean     4.280410    0.528741
std      0.384592    0.364245
min      3.390000   -0.460000
25%      3.975000    0.260000
50%      4.310000    0.510000
75%      4.580000    0.810000
max      5.140000    1.320000


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 732 entries, 1 to 732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype                          
---  ------      --------------  -----                          
 0   hoog_water  707 non-null    float64                        
 1   hoog_tijd   707 non-null    datetime64[ns, Europe/Brussels]
 2   laag_water  707 non-null    float64                        
 3   laag_tijd   707 non-null    datetime64[ns, Europe/Brussels]
dtypes: datetime64[ns, Europe/Brussels](2), float64(2)
memory usage: 23.0 KB


In [32]:
# backup/store the data into a CSV file
df.to_csv('getijden.csv', index=0)

## we now need to filter for dates that allow to build the castle during low tide

In [33]:
# Filter the data to get rows where low tide is between 11:00 and 13:00
# in past experience we saw when predicted low tide ~12u20 CEST means water reach castle ~16u00 4/okt/2015 (event 10:00 tot 18:00)
# in the past we saw when predicted low tide ~11u56 CEST means water reach castle ~16u00 3/sep/2011
low_tide_filter = (df['laag_tijd'].dt.hour >= 11) & (df['laag_tijd'].dt.hour < 13)

In [34]:
# Filter voor maanden tussen mei en oktober (maanden 5 t/m 10) 
# maar negeer de maanden juli (7) en augustus (8)
month_filter = (df['laag_tijd'].dt.month >= 5) & (df['laag_tijd'].dt.month <= 10) & ~(df['laag_tijd'].dt.month.isin([7, 8]))

In [35]:
# Drop NaN values from 'laag_tijd' and get unique years
unique_years = df['laag_tijd'].dropna().dt.year.unique().tolist()

In [36]:
# Fetch Belgian holidays for the unique years
belgian_holidays = Belgium(years=unique_years)

In [37]:
belgian_holidays

{datetime.date(2024, 1, 1): 'Nieuwjaar', datetime.date(2024, 3, 31): 'Pasen', datetime.date(2024, 4, 1): 'Paasmaandag', datetime.date(2024, 5, 1): 'Dag van de Arbeid', datetime.date(2024, 5, 9): 'O. L. H. Hemelvaart', datetime.date(2024, 5, 19): 'Pinksteren', datetime.date(2024, 5, 20): 'Pinkstermaandag', datetime.date(2024, 7, 21): 'Nationale feestdag', datetime.date(2024, 8, 15): 'O. L. V. Hemelvaart', datetime.date(2024, 11, 1): 'Allerheiligen', datetime.date(2024, 11, 11): 'Wapenstilstand', datetime.date(2024, 12, 25): 'Kerstmis'}

In [38]:
# Convert the keys (dates) of the belgian_holidays dictionary to a list
feestdagen = [str(date) for date in belgian_holidays.keys()]

In [39]:
feestdag_filter = df['laag_tijd'].dt.date.astype(str).isin(feestdagen)

In [40]:
# Filter voor enkel weekenddagen: zaterdag (5) of zondag (6)
weekend_filter = df['laag_tijd'].dt.weekday.isin([5, 6])

In [41]:
# Combineer de filters met een OR-operatie
weekend_feestdag = weekend_filter | feestdag_filter

In [42]:
# Combine all the filters
combined_filter = low_tide_filter & month_filter & weekend_feestdag

In [43]:
# Get the days that satisfy all conditions = best days
df[combined_filter]

,hoog_water,hoog_tijd,laag_water,laag_tijd
265,4.66,2024-05-12 04:34:00+02:00,0.31,2024-05-12 11:11:00+02:00
503,4.58,2024-09-08 04:40:00+02:00,0.50,2024-09-08 11:05:00+02:00
531,4.90,2024-09-22 04:27:00+02:00,0.22,2024-09-22 11:03:00+02:00


In [46]:
max_length = max([len(row['hoog_tijd'].strftime("%A %d %B %Y")) for _, row in df[combined_filter].iterrows()])

for index, row in df[combined_filter].iterrows():
    date_str = row['hoog_tijd'].strftime("%A %d %B %Y")
    time_str = row['laag_tijd'].strftime("%Hu%M")
    print(f"{date_str:<{max_length}} (met laag water om {time_str})")

Sunday 12 May 2024       (met laag water om 11u11)
Sunday 08 September 2024 (met laag water om 11u05)
Sunday 22 September 2024 (met laag water om 11u03)
